In [40]:
import scipy.io as scipy
import tensorflow as tf

In [7]:
import numpy as np
import random
import joblib


In [4]:
data = scipy.loadmat("./dataset_lab_276_ul.mat")

In [12]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'csiu_lab', 'label_lab'])

In [17]:
data['csiu_lab'].shape
csi_lab = data['csiu_lab']

In [18]:
data['label_lab'].shape
labels_lab = data['label_lab']

In [20]:
csi_lab[0][0][0][0]

(7.951694273219296+2.3855082819657887j)

In [21]:
csi_lab.shape

(200, 30, 3, 5520)

In [22]:
dummy = np.zeros(shape = (200, 30, 3, 5520))
for timestep in range(200):
    for antenna in range(30):
        for subcar in range(3):
            for sample in range(5520):
                dummy[timestep][antenna][subcar][sample] = abs(csi_lab[timestep][antenna][subcar][sample])

In [23]:
dummy.shape

(200, 30, 3, 5520)

In [24]:
dummy[0][0][0][0]

8.301812547755816

In [25]:
import numpy as np

# data = np.zeros(shape = (200, 3, 30, 2760))
csi = np.zeros(shape = (5520, 200, 90))

for sample in range(5520):
  for timestep in range(200):
    for antenna in range(30):
      for subcar in range(3):
        csi[sample][timestep][3*antenna+subcar] = dummy[timestep][antenna][subcar][sample]

In [26]:
csi.shape

(5520, 200, 90)

In [36]:
labels_lab.shape

(5520, 1)

In [30]:
labels_lab = np.array(labels_lab)-1

In [33]:
labels_lab.min()

0

In [38]:
temp = list(zip(csi, labels_lab))
random.shuffle(temp)
csi, labels_lab = zip(*temp)
csi, labels_lab = list(csi), list(labels_lab)

In [42]:
y_train = tf.constant(labels_lab[:4140])
y_test = tf.constant(labels_lab[4140:5520])
x_train = (csi[:4140])
x_test = (csi[4140:5520])

In [45]:
np.shape(y_train)

TensorShape([4140, 1])

In [136]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense, GlobalMaxPooling1D, Masking, Input
model_1 = models.Sequential([
    # layers.Masking(mask_value = 0.0, input_shape = (None,90), ragged = True),
    # layers.Input([None, 90], dtype=tf.float16, ragged=True),
    layers.BatchNormalization(),
    layers.Conv1D(filters=32, kernel_size=(4), activation='relu', input_shape = (200, 90)),
    layers.MaxPooling1D((2)),

    layers.Conv1D(filters=64, kernel_size=(4), activation='relu'),
    layers.MaxPooling1D((2)),
    
    layers.Conv1D(filters=128, kernel_size=(4), activation='relu'),
    layers.MaxPooling1D((2)),
    
    layers.GlobalMaxPooling1D(),

    layers.Dense(128, activation='relu'),
    layers.Dense(276, activation='softmax')
])

model_1.compile(loss="sparse_categorical_crossentropy", 
                optimizer=tf.keras.optimizers.legacy.Adam(),
                metrics=["accuracy"])

In [137]:
model_1.fit(np.array(x_train), np.array(y_train), epochs = 20)

Epoch 1/20
130/130 [==============================] - 2s 10ms/step - loss: 5.1795 - accuracy: 0.0128
Epoch 2/20
130/130 [==============================] - 1s 10ms/step - loss: 4.2572 - accuracy: 0.0461
Epoch 3/20
130/130 [==============================] - 1s 11ms/step - loss: 3.8409 - accuracy: 0.0894
Epoch 4/20
130/130 [==============================] - 1s 11ms/step - loss: 3.5204 - accuracy: 0.1374
Epoch 5/20
130/130 [==============================] - 1s 10ms/step - loss: 3.2162 - accuracy: 0.1913
Epoch 6/20
130/130 [==============================] - 1s 10ms/step - loss: 2.9289 - accuracy: 0.2430
Epoch 7/20
130/130 [==============================] - 1s 11ms/step - loss: 2.5827 - accuracy: 0.3070
Epoch 8/20
130/130 [==============================] - 1s 10ms/step - loss: 2.3594 - accuracy: 0.3519
Epoch 9/20
130/130 [==============================] - 1s 10ms/step - loss: 2.0091 - accuracy: 0.4333
Epoch 10/20
130/130 [==============================] - 1s 10ms/step - loss: 1.6155 - accura

In [138]:
model_1.evaluate(np.array(x_test), np.array(y_test))

44/44 [==============================] - 0s 4ms/step - loss: 1.3054 - accuracy: 0.6246


[1.3053652048110962, 0.6246376633644104]

In [142]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense, GlobalMaxPooling1D, Masking, Input

model_2 = models.Sequential([
    # layers.Masking(mask_value = 0.0, input_shape = (None,90), ragged = True),
    # layers.Input([None, 90], dtype=tf.float16, ragged=True),
    layers.BatchNormalization(),
    layers.Conv1D(filters=32, kernel_size=(7), activation='relu', input_shape = (200, 90)),
    # layers.Conv1D(filters=32, kernel_size=(4), activation='relu', input_shape = (200, 90)),
    layers.MaxPooling1D((2)),

    layers.Conv1D(filters=64, kernel_size=(4), activation='relu'),
    layers.Conv1D(filters=64, kernel_size=(4), activation='relu'),
    layers.MaxPooling1D((2)),
    
    layers.Conv1D(filters=128, kernel_size=(4), activation='relu'),
    layers.Conv1D(filters=128, kernel_size=(4), activation='relu'),
    layers.MaxPooling1D((2)),
    
    layers.GlobalMaxPooling1D(),

    layers.Dense(128, activation='relu'),
    layers.Dense(276, activation='softmax')
])

model_2.compile(loss="sparse_categorical_crossentropy", 
                optimizer=tf.keras.optimizers.legacy.Adam(),
                metrics=["accuracy"])

In [143]:
model_2.fit(np.array(x_train), np.array(y_train), epochs = 20)

Epoch 1/20
130/130 [==============================] - 3s 17ms/step - loss: 5.1832 - accuracy: 0.0126
Epoch 2/20
130/130 [==============================] - 2s 18ms/step - loss: 4.3447 - accuracy: 0.0396
Epoch 3/20
130/130 [==============================] - 2s 18ms/step - loss: 3.8371 - accuracy: 0.1034
Epoch 4/20
130/130 [==============================] - 2s 17ms/step - loss: 3.4868 - accuracy: 0.1623
Epoch 5/20
130/130 [==============================] - 2s 17ms/step - loss: 3.2375 - accuracy: 0.1942
Epoch 6/20
130/130 [==============================] - 2s 17ms/step - loss: 2.9922 - accuracy: 0.2403
Epoch 7/20
130/130 [==============================] - 2s 19ms/step - loss: 2.7411 - accuracy: 0.2884
Epoch 8/20
130/130 [==============================] - 2s 17ms/step - loss: 2.5121 - accuracy: 0.3304
Epoch 9/20
130/130 [==============================] - 2s 17ms/step - loss: 2.3138 - accuracy: 0.3773
Epoch 10/20
130/130 [==============================] - 2s 17ms/step - loss: 2.0384 - accura

In [144]:
model_2.evaluate(np.array(x_test), np.array(y_test))

44/44 [==============================] - 0s 6ms/step - loss: 1.4880 - accuracy: 0.6275


[1.4880174398422241, 0.6275362372398376]

In [145]:
model_2.summary()

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_12 (Ba  (None, 200, 90)           360       
 tchNormalization)                                               
                                                                 
 conv1d_96 (Conv1D)          (None, 194, 32)           20192     
                                                                 
 max_pooling1d_82 (MaxPooli  (None, 97, 32)            0         
 ng1D)                                                           
                                                                 
 conv1d_97 (Conv1D)          (None, 94, 64)            8256      
                                                                 
 conv1d_98 (Conv1D)          (None, 91, 64)            16448     
                                                                 
 max_pooling1d_83 (MaxPooli  (None, 45, 64)          

In [146]:
model_2.save('model_2.h5')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
